<a href="https://colab.research.google.com/github/Vilmo18/Fine-Tuning-LLMs/blob/main/NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flair
!pip install datasets

ERROR: Invalid requirement: 'flair!pip'


In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger

In [ ]:
tagger = SequenceTagger.load('flair/ner-english-ontonotes-large')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


2024-06-25 15:52:21,982 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


In [ ]:
included_tags = {
    "EVENT", "FAC", "GPE", "LANGUAGE", "LAW", "LOCATION", "NORP",
    "ORG", "PERSON", "PRODUCT", "WORK_OF_ART"
}

excluded_tags = {
    "CARDINAL", "DATE", "MONEY", "ORDINAL", "PERCENT", "QUANTITY", "TIME"
}

def anonymize_entities(text):
    sentence = Sentence(text)
    tagger.predict(sentence)

    token_replacements = {}

    for entity in sentence.get_spans('ner'):
        if entity.tag in included_tags:
            #Replace the entity with its tag
            token_replacements[(entity.start_position, entity.end_position)] = f"[{entity.tag}]"

    #Create anonymized text by replacing entities with their tags
    anonymized_text = ''
    last_idx = 0

    for (start, end), replacement in sorted(token_replacements.items()):
        anonymized_text += text[last_idx:start] + replacement
        last_idx = end

    anonymized_text += text[last_idx:]

    return anonymized_text


text = "Francesco Totti threatens to quit the Italian Players' Association . The Italy and Roma striker is upset over the early start to the new season . He says: We are the principal actors, but ... are never listened to."
anonymized_text = anonymize_entities(text)
print(anonymized_text)


[PERSON] threatens to quit [ORG] . The [GPE] and [GPE] striker is upset over the early start to the new season . He says: We are the principal actors, but ... are never listened to.


In [ ]:
from datasets import load_dataset
ds = load_dataset("abisee/cnn_dailymail", "3.0.0")

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [ ]:
with open("article.txt", "w") as articles , open("highlights.txt", "w") as highlights:
    for it in ds["train"]:
        articles.write(it["article"] + "\n")
        highlights.write(it["highlights"] + "\n")

In [ ]:
!pip install sentencepiece

In [ ]:
!wget https://raw.githubusercontent.com/google/sentencepiece/master/data/botchan.txt

--2024-06-25 15:53:02--  https://raw.githubusercontent.com/google/sentencepiece/master/data/botchan.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 278779 (272K) [text/plain]
Saving to: ‘botchan.txt.1’

botchan.txt.1       100%[===================>] 272.25K  --.-KB/s    in 0.03s   

2024-06-25 15:53:02 (9.36 MB/s) - ‘botchan.txt.1’ saved [278779/278779]



In [ ]:
import sentencepiece as spm

# train sentencepiece model from `botchan.txt` and makes `m.model` and `m.vocab`
# `m.vocab` is just a reference. not used in the segmentation.
spm.SentencePieceTrainer.train('--input=highlights.txt')

# makes segmenter instance and loads the model file (m.model)
sp = spm.SentencePieceProcessor()
sp.load('m.model')

# encode: text => id
print(sp.encode_as_pieces('This is a test'))
print(sp.encode_as_ids('This is a test'))

# decode: id => text
print(sp.decode_pieces(['▁This', '▁is', '▁a', '▁t', 'est']))
print(sp.decode_ids([208, 31, 9, 434, 601]))

In [ ]:
# encode: text => id
print(sp.encode_as_pieces(anonymized_text))

['▁', '[', 'P', 'E', 'R', 'S', 'O', 'N', ']', '▁threat', 'en', 's', '▁to', '▁', 'qui', 't', '▁', '[', 'O', 'R', 'G', ']', '▁.', '▁The', '▁', '[', 'G', 'P', 'E', ']', '▁and', '▁', '[', 'G', 'P', 'E', ']', '▁striker', '▁is', '▁up', 's', 'et', '▁over', '▁the', '▁early', '▁start', '▁to', '▁the', '▁new', '▁season', '▁.', '▁He', '▁says', ':', '▁We', '▁are', '▁the', '▁p', 'ri', 'n', 'ci', 'p', 'al', '▁a', 'ctor', 's', ',', '▁but', '▁', '...', '▁are', '▁never', '▁list', 'en', 'ed', '▁to', '.']


In [ ]:
anonymized_text

'[PERSON] threatens to quit [ORG] . The [GPE] and [GPE] striker is upset over the early start to the new season . He says: We are the principal actors, but ... are never listened to.'